In [2]:
# Imports
import os
import json
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import re


# For deep learning
import torch
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.ops import box_iou

from torchmetrics.detection.mean_ap import MeanAveragePrecision
from sklearn.metrics import precision_recall_fscore_support
import numpy as np


# For augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Own package imports
os.chdir('/home/naro/projects/Rumex')

from utils.data_inspection import *
from src.augmentation import *
from src.dataset import *
from utils.viz_utils import *
from utils.data_utils import *
from src.model_factory import *
from src.evaluate import *
from src.train import *
from src.inference import predict_and_visualize_image, load_best_model
from utils.fiftyone_utils import *
from src.hyperparameters_tuning import *

VIZ = False

In [ ]:
# Evaluate it using the map50 function

In [ ]:
# Implement a new evaluation function here, then deploy it to the pipeline